뉴스 기사 제목 데이터에 대한 이해
약 15년 동안 발행되었던 뉴스 기사 제목을 모아놓은 영어 데이터를 아래 링크에서 다운받을 수 있습니다.

데이터 다운 받으면 됨

링크 : https://www.kaggle.com/therohk/million-headlines

In [48]:
import pandas as pd
import urllib.request
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation


In [49]:
data = pd.read_csv('abcnews-date-text.csv', error_bad_lines=False)
print('뉴스 제목 개수 :',len(data))

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


뉴스 제목 개수 : 1244184


In [50]:
print(data.head(5))

   publish_date                                      headline_text
0      20030219  aba decides against community broadcasting lic...
1      20030219     act fire witnesses must be aware of defamation
2      20030219     a g calls for infrastructure protection summit
3      20030219           air nz staff in aust strike for pay rise
4      20030219      air nz strike to affect australian travellers


기사의 헤드라인만 필요하기 때문에 text라는 이름으로 따로 저장

In [51]:
text = data[['headline_text']]
text.head(5)

,headline_text
0,aba decides against community broadcasting lic...
1,act fire witnesses must be aware of defamation
2,a g calls for infrastructure protection summit
3,air nz staff in aust strike for pay rise
4,air nz strike to affect australian travellers


## 텍스트 전처리
불용어 제거, 표제어 추출, 길이가 짧은 단어 제거라는 세 가지 전처리 기법
NLTK의 word_tokenize를 통해 단어 토큰화를 수행합니다.

In [52]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [53]:
text['headline_text'] = text.apply(lambda row: nltk.word_tokenize(row['headline_text']), axis=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [54]:
print(text.head(5))

                                       headline_text
0  [aba, decides, against, community, broadcastin...
1  [act, fire, witnesses, must, be, aware, of, de...
2  [a, g, calls, for, infrastructure, protection,...
3  [air, nz, staff, in, aust, strike, for, pay, r...
4  [air, nz, strike, to, affect, australian, trav...


NLTK가 제공하는 영어 불용어를 통해서 text 데이터로부터 불용어를 제거

In [55]:
stop_words = stopwords.words('english')
text['headline_text'] = text['headline_text'].apply(lambda x: [word for word in x if word not in (stop_words)])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [56]:
print(text.head(5))

                                       headline_text
0   [aba, decides, community, broadcasting, licence]
1    [act, fire, witnesses, must, aware, defamation]
2     [g, calls, infrastructure, protection, summit]
3          [air, nz, staff, aust, strike, pay, rise]
4  [air, nz, strike, affect, australian, travellers]


against, be, of, a, in, to 등의 단어가 제거 된걸 확인 할 수 있다.

In [60]:
 nltk.download('wordnet')
 nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

표제어 추출을 수행합니다. 
표제어 추출로 3인칭 단수 표현을 1인칭으로 바꾸고, 과거 현재형 동사를 현재형으로 바꿉니다.

In [61]:
text['headline_text'] = text['headline_text'].apply(lambda x: [WordNetLemmatizer().lemmatize(word, pos='v') for word in x])
print(text.head(5))

                                       headline_text
0       [aba, decide, community, broadcast, licence]
1      [act, fire, witness, must, aware, defamation]
2      [g, call, infrastructure, protection, summit]
3          [air, nz, staff, aust, strike, pay, rise]
4  [air, nz, strike, affect, australian, travellers]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


길이가 3이하인 단어에 대해서 제거하는 작업을 수행합니다.

In [63]:
tokenized_doc = text['headline_text'].apply(lambda x: [word for word in x if len(word) > 3])
print(tokenized_doc[:5])

0       [decide, community, broadcast, licence]
1      [fire, witness, must, aware, defamation]
2    [call, infrastructure, protection, summit]
3                   [staff, aust, strike, rise]
4      [strike, affect, australian, travellers]
Name: headline_text, dtype: object


## TF-IDF 행렬 만들기

In [64]:
# 역토큰화 (토큰화 작업을 되돌림)
detokenized_doc = []
for i in range(len(text)):
    t = ' '.join(tokenized_doc[i])
    detokenized_doc.append(t)

# 다시 text['headline_text']에 재저장
text['headline_text'] = detokenized_doc

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [65]:
text['headline_text'][:5]

0       decide community broadcast licence
1       fire witness must aware defamation
2    call infrastructure protection summit
3                   staff aust strike rise
4      strike affect australian travellers
Name: headline_text, dtype: object

In [66]:
# 상위 1,000개의 단어를 보존 
vectorizer = TfidfVectorizer(stop_words='english', max_features= 1000)
X = vectorizer.fit_transform(text['headline_text'])

In [67]:
# TF-IDF 행렬의 크기 확인
print('TF-IDF 행렬의 크기 :',X.shape)

TF-IDF 행렬의 크기 : (1244184, 1000)


In [68]:
lda_model = LatentDirichletAllocation(n_components=10,learning_method='online',random_state=777,max_iter=1)
lda_top = lda_model.fit_transform(X)
print(lda_model.components_)
print(lda_model.components_.shape) 

[[1.00005784e-01 1.00000401e-01 1.00000837e-01 ... 1.00008966e-01
  1.00003001e-01 1.00003989e-01]
 [1.00001271e-01 1.00000170e-01 1.00000702e-01 ... 1.00009027e-01
  1.00004822e-01 6.11677308e+02]
 [1.00001893e-01 1.00000747e-01 5.29631843e+02 ... 1.00004665e-01
  1.00003828e-01 1.00003730e-01]
 ...
 [1.00006258e-01 1.00000570e-01 1.00000388e-01 ... 1.00006276e-01
  1.00001666e-01 1.00008566e-01]
 [1.00000280e-01 1.00000135e-01 1.00001609e-01 ... 1.00003391e-01
  1.00001003e-01 1.00006488e-01]
 [1.03272231e+02 1.00000349e-01 1.00001881e-01 ... 1.00005116e-01
  1.00004106e-01 1.00006425e-01]]
(10, 1000)


In [69]:
# 단어 집합. 1,000개의 단어가 저장됨.
terms = vectorizer.get_feature_names()

def get_topics(components, feature_names, n=5):
    for idx, topic in enumerate(components):
        print("Topic %d:" % (idx+1), [(feature_names[i], topic[i].round(2)) for i in topic.argsort()[:-n - 1:-1]])

get_topics(lda_model.components_,terms)

Topic 1: [('australia', 20556.0), ('sydney', 11219.29), ('melbourne', 8765.73), ('kill', 6646.06), ('court', 6004.12)]
Topic 2: [('coronavirus', 41719.62), ('covid', 28960.68), ('government', 9793.89), ('change', 7576.98), ('home', 7457.74)]
Topic 3: [('south', 7102.98), ('death', 6825.39), ('speak', 5402.31), ('care', 4521.48), ('interview', 4058.71)]
Topic 4: [('donald', 8536.49), ('restrictions', 6456.4), ('world', 6320.61), ('state', 6087.5), ('water', 4219.67)]
Topic 5: [('vaccine', 8040.87), ('open', 6915.17), ('coast', 5990.55), ('warn', 5472.84), ('morrison', 5247.93)]
Topic 6: [('trump', 14878.13), ('charge', 7717.96), ('health', 6836.95), ('murder', 6663.45), ('house', 6624.13)]
Topic 7: [('australian', 13885.88), ('queensland', 13373.53), ('record', 9037.88), ('test', 7713.9), ('help', 5922.05)]
Topic 8: [('case', 13146.83), ('police', 11143.1), ('live', 7528.03), ('border', 6855.54), ('tasmania', 5664.64)]
Topic 9: [('victoria', 11777.5), ('school', 6009.78), ('attack', 550

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


새로운 TASK

In [72]:
tokenized_doc[:5]

0       [decide, community, broadcast, licence]
1      [fire, witness, must, aware, defamation]
2    [call, infrastructure, protection, summit]
3                   [staff, aust, strike, rise]
4      [strike, affect, australian, travellers]
Name: headline_text, dtype: object

In [73]:
from gensim import corpora
dictionary = corpora.Dictionary(tokenized_doc)
corpus = [dictionary.doc2bow(text) for text in tokenized_doc]
print(corpus[1]) # 수행된 결과에서 두번째 뉴스 출력. 첫번째 문서의 인덱스는 0

[(4, 1), (5, 1), (6, 1), (7, 1), (8, 1)]


In [74]:
print(dictionary[66])

dismiss


In [75]:
len(dictionary)

92311

In [76]:
import gensim
NUM_TOPICS = 20 # 20개의 토픽, k=20
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.170*"covid" + 0.039*"fire" + 0.038*"coronavirus" + 0.031*"lockdown"')
(1, '0.052*"court" + 0.051*"charge" + 0.040*"face" + 0.040*"murder"')
(2, '0.080*"australian" + 0.045*"restrictions" + 0.025*"perth" + 0.023*"water"')
(3, '0.055*"melbourne" + 0.040*"people" + 0.031*"federal" + 0.030*"premier"')
(4, '0.043*"kill" + 0.032*"concern" + 0.027*"amid" + 0.027*"hospital"')
(5, '0.089*"queensland" + 0.057*"test" + 0.029*"adelaide" + 0.020*"coronavirus"')
(6, '0.041*"first" + 0.040*"death" + 0.034*"police" + 0.027*"time"')
(7, '0.032*"return" + 0.029*"pandemic" + 0.027*"work" + 0.027*"former"')
(8, '0.040*"donald" + 0.039*"live" + 0.030*"school" + 0.029*"scott"')
(9, '0.089*"case" + 0.068*"victoria" + 0.034*"coronavirus" + 0.027*"quarantine"')
(10, '0.079*"trump" + 0.031*"could" + 0.025*"women" + 0.019*"president"')
(11, '0.074*"sydney" + 0.060*"government" + 0.030*"national" + 0.028*"rise"')
(12, '0.044*"state" + 0.042*"coast" + 0.027*"rule" + 0.026*"gold"')
(13, '0.054*"find" + 0.053

In [77]:
print(ldamodel.print_topics())

[(0, '0.170*"covid" + 0.039*"fire" + 0.038*"coronavirus" + 0.031*"lockdown" + 0.030*"house" + 0.028*"year" + 0.024*"report" + 0.023*"record" + 0.022*"world" + 0.016*"three"'), (1, '0.052*"court" + 0.051*"charge" + 0.040*"face" + 0.040*"murder" + 0.036*"brisbane" + 0.033*"market" + 0.023*"accuse" + 0.021*"release" + 0.020*"hear" + 0.020*"police"'), (2, '0.080*"australian" + 0.045*"restrictions" + 0.025*"perth" + 0.023*"water" + 0.021*"fight" + 0.020*"royal" + 0.018*"fall" + 0.017*"commission" + 0.016*"deal" + 0.015*"build"'), (3, '0.055*"melbourne" + 0.040*"people" + 0.031*"federal" + 0.030*"premier" + 0.025*"back" + 0.020*"fund" + 0.018*"officer" + 0.017*"abuse" + 0.017*"council" + 0.017*"meet"'), (4, '0.043*"kill" + 0.032*"concern" + 0.027*"amid" + 0.027*"hospital" + 0.027*"crash" + 0.020*"flight" + 0.017*"continue" + 0.017*"media" + 0.016*"violence" + 0.016*"doctor"'), (5, '0.089*"queensland" + 0.057*"test" + 0.029*"adelaide" + 0.020*"coronavirus" + 0.019*"final" + 0.018*"beach" + 0.

시각화 하기

In [78]:
pip install pyLDAvis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.7 MB 18.3 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136898 sha256=a88e9f5bd7b7cdbd53c784c8d889f75524b4ebb247abbc33a46abb07365dbd73
  Stored in directory: /root/.cache/pip/wheels/c9/21/f6/17bcf2667e8a68532ba2fbf6d5c72fdf4c7f7d9abfa4852d2f
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1310 sha256=2b20e4560f0bb17e29728c51eb5b477ef642b6f60c1972749dbace4709fa8897
  Stored in directory: /root/.cache/pip/wheels/46/ef/c3/157e41f5ee1372d1be90b09f74f82b10e391eaacca8f22d33e
Successfully built pyLDAvis sklearn


In [79]:
import pyLDAvis.gensim_models

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable


In [80]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(ldamodel, corpus, dictionary)
pyLDAvis.display(vis)

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


문서 별 토픽 분포 보기

In [81]:
for i, topic_list in enumerate(ldamodel[corpus]):
    if i==5:
        break
    print(i,'번째 문서의 topic 비율은',topic_list)

0 번째 문서의 topic 비율은 [(17, 0.21), (19, 0.61)]
1 번째 문서의 topic 비율은 [(0, 0.175), (13, 0.175), (16, 0.5083333)]
2 번째 문서의 topic 비율은 [(3, 0.56184846), (7, 0.25815153)]
3 번째 문서의 topic 비율은 [(9, 0.21), (11, 0.21), (12, 0.41)]
4 번째 문서의 topic 비율은 [(2, 0.21), (10, 0.21), (12, 0.21), (15, 0.21)]


In [82]:
def make_topictable_per_doc(ldamodel, corpus):
    topic_table = pd.DataFrame()

    # 몇 번째 문서인지를 의미하는 문서 번호와 해당 문서의 토픽 비중을 한 줄씩 꺼내온다.
    for i, topic_list in enumerate(ldamodel[corpus]):
        doc = topic_list[0] if ldamodel.per_word_topics else topic_list            
        doc = sorted(doc, key=lambda x: (x[1]), reverse=True)
        # 각 문서에 대해서 비중이 높은 토픽순으로 토픽을 정렬한다.
        # EX) 정렬 전 0번 문서 : (2번 토픽, 48.5%), (8번 토픽, 25%), (10번 토픽, 5%), (12번 토픽, 21.5%), 
        # Ex) 정렬 후 0번 문서 : (2번 토픽, 48.5%), (8번 토픽, 25%), (12번 토픽, 21.5%), (10번 토픽, 5%)
        # 48 > 25 > 21 > 5 순으로 정렬이 된 것.

        # 모든 문서에 대해서 각각 아래를 수행
        for j, (topic_num, prop_topic) in enumerate(doc): #  몇 번 토픽인지와 비중을 나눠서 저장한다.
            if j == 0:  # 정렬을 한 상태이므로 가장 앞에 있는 것이 가장 비중이 높은 토픽
                topic_table = topic_table.append(pd.Series([int(topic_num), round(prop_topic,4), topic_list]), ignore_index=True)
                # 가장 비중이 높은 토픽과, 가장 비중이 높은 토픽의 비중과, 전체 토픽의 비중을 저장한다.
            else:
                break
    return(topic_table)

In [ ]:
topictable = make_topictable_per_doc(ldamodel, corpus)
topictable = topictable.reset_index() # 문서 번호을 의미하는 열(column)로 사용하기 위해서 인덱스 열을 하나 더 만든다.
topictable.columns = ['문서 번호', '가장 비중이 높은 토픽', '가장 높은 토픽의 비중', '각 토픽의 비중']
topictable[:10]